<a href="https://colab.research.google.com/github/vit050587/neural_networks/blob/master/KVA_INN_HW_07(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 7. Детектирование объектов

Сделайте краткий обзор любой научной работы, посвящённой алгоритму для object detection, не рассматривавшемуся на уроке. Проведите анализ: чем отличается выбранная вами архитектура нейронной сети от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении этой архитектуры на практике?

In [33]:
#!pip install tensorflow
#!pip install tensorflow==2.8.0
#!pip install tensorflow-gpu==2.8.0
#!pip install tensorflow==2.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
#!pip install keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0
#!pip install keras numpy pillow scipy h5py matplotlib opencv-python keras-resnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Using cached numpy-1.19.3.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Pillow-7.0.0.tar.gz (38.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [35]:
#!pip install imageai
!pip install imageai==2.1.6
#!pip install imageai --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imageai-2.1.6-py3-none-any.whl (160 kB)
  Using cached numpy-1.19.3.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [36]:
#!pip install cython pillow>=7.0.0 numpy>=1.18.1 opencv-python>=4.1.2 torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cpu torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cpu pytest==7.1.3 tqdm==4.64.1 scipy>=1.7.3 matplotlib>=3.4.3 mock==4.0.3

In [37]:
#!pip install cython pillow>=7.0.0 numpy>=1.18.1 opencv-python>=4.1.2 torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cu102 torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cu102 pytest==7.1.3 tqdm==4.64.1 scipy>=1.7.3 matplotlib>=3.4.3 mock==4.0.3

In [38]:
#!pip install pycocotools@git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

In [39]:
#!pip install imageai --upgrade

In [40]:
import tensorflow as tf
tf.keras.layers.BatchNormalization()

In [41]:
from imageai.Detection import VideoObjectDetection

In [42]:
from keras.layers import ELU, PReLU, LeakyReLU
from keras.layers import BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense

In [ ]:
import numpy as np
import os
import shutil
from pathlib import Path
from PIL import Image

from keras.layers import BatchNormalization
from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection

# Prepare dataset

In [44]:
root_annots_path = '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/annotations'
root_images_path = '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/images'

annots_path = sorted([i for i in Path(root_annots_path).glob('*.xml')])
images_path = sorted([i for i in Path(root_images_path).glob('*.png')])

n_imgs = len(images_path)
print('Количество картинок: ', n_imgs)

Количество картинок:  81


In [45]:
classes = np.array(["white-pawn", "black-pawn",
                    "white-bishop", "black-bishop",
                    "white-knight", "black-knight",
                    "white-king", "black-king",
                    "white-queen", "black-queen",
                    "white-rook", "black-rook"])

In [46]:
with open(annots_path[50], 'r') as f:
    print(f.read())


<annotation>
    <folder>images</folder>
    <filename>chess54.png</filename>
    <size>
        <width>400</width>
        <height>225</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>black-rook</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>203</xmin>
            <ymin>144</ymin>
            <xmax>259</xmax>
            <ymax>213</ymax>
        </bndbox>
    </object>
    <object>
        <name>white-pawn</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>296</xmin>
            <ymin>151</ymin>
            <xmax>359</xmax>
            <ymax>205</ymax>
        </bndbox>
    </object>
    <object>
        <name>black-knight</name>
        <pose>Unspecified</pose>
        <truncated>0</trunca

In [47]:
# create foldes for train and test
os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/train/images', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/train/annotations', exist_ok=True)

os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/validation/images', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/validation/annotations', exist_ok=True)

os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/test/images', exist_ok=True)
os.makedirs('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/test/annotations', exist_ok=True)

In [48]:
# делим картинки на трейн, валидацию и тест
n_split = n_imgs // 20


for i, (annot_path, img_path) in enumerate(zip(annots_path, images_path)):    
    if i > n_imgs:
        break
    # train-val-test split
    if i < n_split:
        shutil.copy(img_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/test/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/test/annotations/' + annot_path.parts[-1])
    elif n_split <= i < n_split*5:
        shutil.copy(img_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/validation/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/validation/annotations/' + annot_path.parts[-1])
    else:
        shutil.copy(img_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/train/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/train/annotations/' + annot_path.parts[-1])

In [49]:
print('test: ', len(list(Path('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/train/annotations/').glob('*.xml'))))
print('val: ', len(list(Path('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/validation/annotations/').glob('*.xml'))))
print('test: ', len(list(Path('/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/test/annotations/').glob('*.xml'))))

test:  61
val:  16
test:  4


In [50]:
# скачиваем веса предобученной модели
%cd /content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/
!wget wget https://pjreddie.com/media/files/yolov3.weights

/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07
--2023-05-11 15:18:31--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-05-11 15:18:31--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  27.6MB/s    in 8.4s    

2023-05-11 15:18:39 (28.2 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

FINISHED --2023-05-11 15:18:39--
Total wall clock time: 8.8s
Downloaded: 1 files, 237M in 8.4s (28.2 MB/s)


In [51]:
# скачиваем конвертер весов и конфиг https://github.com/qqwweee/keras-yolo3/find/master
!wget wget https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/convert.py
!wget wget https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/yolov3.cfg

--2023-05-11 15:18:45--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-05-11 15:18:45--  https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/convert.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10093 (9.9K) [text/plain]
Saving to: ‘convert.py.1’

convert.py.1        100%[===================>]   9.86K  --.-KB/s    in 0.002s  

2023-05-11 15:18:45 (6.17 MB/s) - ‘convert.py.1’ saved [10093/10093]

FINISHED --2023-05-11 15:18:45--
Total wall clock time: 0.2s
Downloaded: 1 files, 9.9K in 0.002s (6.17 MB/s)
--2023-05-11 15:18:45--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2023-05-11 15:18:45--  h

In [52]:
%run convert.py yolov3.cfg yolov3.weights yolov3.h5

Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_1
Parsing section convolutional_7
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_2
Parsing section convolutional_9
conv2d bn leaky (3, 3, 128, 256)
Parsing section convolutional_10
conv2d bn leaky (1, 1, 256, 128)
Parsing section convolutional_11
conv2d bn leaky (3, 3, 128, 256)
Parsing section shortcut_3
Parsi

None
Saved Keras model to yolov3.h5
Read 62001757 of 62001757.0 from Darknet weights.


# Training on dataset

In [53]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3() # дообучать можно только 'большую' YOLO (разработчики)
trainer.setDataDirectory(data_directory='/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/') # где лежат даные
trainer.setTrainConfig(object_names_array=classes,
                       batch_size=8,
                       num_experiments=20,
                       # дообучаемся на их модели
                       train_from_pretrained_model='/content/drive/MyDrive/Colab Notebooks/neural_networks/lesson07/yolov3.h5',
                       
                       # можно дообучаться со своей модели
                       #train_from_pretrained_model="imageai/data/models/detection_model-ex-009--loss-0024.110.h5")
                       )

NameError: ignored

In [ ]:
trainer.trainModel()

# Evaluating your saved detection models

In [ ]:
# извлекаем путь к последней модели
model_path = sorted(list(Path('/content/chess-piece-dectection/models/').iterdir()))[-1]
model_path

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="/content/chess-piece-dectection/")
metrics = trainer.evaluateModel(model_path=model_path,
                                json_path="/content/chess-piece-dectection/json/detection_config.json",
                                iou_threshold=0.2,
                                object_threshold=0.3,
                                nms_threshold=0.5)

In [ ]:
input_image_path="/content/chess-piece-dectection/test/images/chess0.png"
output_image_path="/content/chess-piece-dectection/test/images/detected.jpg"

In [ ]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(model_path)
detector.setJsonPath("/content/chess-piece-dectection/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(minimum_percentage_probability=60,
                                             input_image=input_image_path,
                                             output_image_path=output_image_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

In [ ]:
Image.open(input_image_path)

In [ ]:
Image.open(output_image_path)